In [193]:
input = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [226]:
from functools import total_ordering

In [208]:
with open('day7.txt', 'r') as f:
    input = f.read()

In [241]:
class Path:
    def __init__(self, typ, name, parent):
        self.typ = typ
        self.name = name
        self.parent = parent
        self.children = []
    
    def add(self, p: Path):
        self.children.append(p)
        
    def find_child(self, p: Path) -> Path:
        for f in self.children:
            if f == p:
                return f
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return f"{self.name} ({self.typ})"
    
    def __eq__(self, other): 
        return self.typ == other.typ and self.name == other.name and self.parent == other.parent
    
    def render(self, depth=0):
        print(f"{' '*depth}- {self}")
        for c in self.children:
            c.render(depth + 1)
            
    @staticmethod
    def walk(node):
        for n in node.children:
            yield n
            for c in Path.walk(n):
                yield c

class Directory(Path):
    def __init__(self, name, parent):
        super().__init__('dir', name, parent)
    
    @property
    def size(self):
        return sum([ x.size for x in self.children ])
        
class File(Path):
    def __init__(self, size, name, parent):
        super().__init__('file', name, parent)
        self._size = int(size)
        
    def __str__(self):
        return f"{self.name} ({self.typ}, size={self.size})"
    
    @property
    def size(self):
        return self._size
    
class Command:
    def __init__(self, line):
        assert Command.is_command(line), f"`{line}` is not a command?"
        cmds = line.split()[1:]
        self.cmd = cmds[0]
        self.arg = cmds[1] if len(cmds) > 1 else None
        
    def __eq__(self, other):
        if isinstance(other, str):
            return self.cmd == other
        return self.cmd == other.cmd
    
    @staticmethod
    def is_command(line):
        return line[0] == "$"

In [245]:
def parse_input(lines, node):
    for i, line in enumerate(lines):
        if Command.is_command(line):
            cmd = Command(line)
            if cmd == "ls":
                continue
            elif cmd == "cd":
                if cmd.arg == '..':
                    return parse_input(lines[i+1:], node.parent)
                return parse_input(lines[i+1:], node.find_child(Directory(cmd.arg, node)))
        elif line.startswith("dir"):
            node.add(Directory(line.split()[-1], node))
        else:
            node.add(File(*line.split(), parent=node))

root = Directory('/', None)
parse_input(input.split('\n')[1:], root)

In [246]:
#root.render()

In [247]:
root.size

46975962

In [251]:
sum([ x.size for x in Path.walk(root) if x.typ == "dir" and x.size < 100000])

1334506

In [259]:
needed = 30000000
unused = 70000000 - root.size
could_delete = [ x for x in Path.walk(root) if x.typ == "dir" and (unused + x.size) >= needed]
could_delete.sort(key=lambda x: x.size)
could_delete[0].size

7421137